In [1]:
import pandas as pd
import csv
import os
import numpy as np
from nltk import pos_tag, word_tokenize
from ingredient_parser.en import parse 
pd.set_option('display.expand_frame_repr', False)


In [12]:
recipes = pd.read_csv('../../data/recipes.csv')
recipes.columns=['recipe_id', 'recipe_tag', 'title', 'rating', 'total_reviews', 'ingredients', 'calories', 'carbs',\
                'protein', 'cholesterol', 'sodium']
recipes.head()

,recipe_id,recipe_tag,title,rating,total_reviews,ingredients,calories,carbs,protein,cholesterol,sodium
0,1,italian,Italian Chicken Cacciatore,5.00,3,"[u'1/4 cup olive oil, divided', u'1 onion, dic...",587,36.5,38.8,117,1707
1,2,italian,Spaghetti Cacio e Pepe,4.58,73,"[u'1 pound spaghetti', u'6 tablespoons olive o...",806,87.8,31.6,54,633
2,3,italian,Easy Caponata,4.67,3,"[u'2 stalks stalk celery, chopped into 1/4-inc...",289,25.2,4.4,0,424
3,4,italian,Chef John's Lemon Ice,4.64,10,"[u'4 cups cold water', u'7/8 cup white sugar',...",94,24.8,0.1,0,4
4,5,italian,Focaccia di Recco,4.60,17,"[u'2 cups all-purpose flour', u'1/2 cup cold w...",186,24.1,4.7,7,327


In [13]:
# First genereate the array of ingredients
recipes['ingredients'] = recipes['ingredients'].apply(lambda a: a.replace("[", "").replace("]", "").replace("u'","")\
                                                      .split("',"))

recipes.head()

,recipe_id,recipe_tag,title,rating,total_reviews,ingredients,calories,carbs,protein,cholesterol,sodium
0,1,italian,Italian Chicken Cacciatore,5.00,3,"[1/4 cup olive oil, divided, 1 onion, diced, ...",587,36.5,38.8,117,1707
1,2,italian,Spaghetti Cacio e Pepe,4.58,73,"[1 pound spaghetti, 6 tablespoons olive oil, ...",806,87.8,31.6,54,633
2,3,italian,Easy Caponata,4.67,3,"[2 stalks stalk celery, chopped into 1/4-inch ...",289,25.2,4.4,0,424
3,4,italian,Chef John's Lemon Ice,4.64,10,"[4 cups cold water, 7/8 cup white sugar, 2 t...",94,24.8,0.1,0,4
4,5,italian,Focaccia di Recco,4.60,17,"[2 cups all-purpose flour, 1/2 cup cold water...",186,24.1,4.7,7,327


In [14]:
# This makes use of the model from NY Times that will take the ingredients and predict the base root ingredient
# Example: 1/4 cup olive oil, will be predicted to be as olive oil (hopefully)
# Repo found here: https://github.com/NYTimes/ingredient-phrase-tagger
def process_ingredients(data):
    data_to_parse = np.asarray(data)
    parsed = []
    list_lol = ["sliced","diced","chopped","dried","minced","crushed","grated","shredded","marinated","sweetened","crumbled", "cooked", "cubed","smoked", "uncooked", "toasted"]
    for ingre in data_to_parse:
        str_parsed = parse(ingre).get("name")
        string = str_parsed.split(",", 1)[0]
        for lol in list_lol:
            if lol in string:
                string = string.split(lol, 1)[1]
        # if "sliced" in somestring: 
        #     string = string.split("sliced", 1)[1]
        # if "sliced" in somestring:
        #     string = string.split("diced", 1)[1]
        # if "sliced" in somestring:
        #     string = string.split("chopped", 1)[1]
        # if "sliced" in somestring:
        #     string = string.split("dried", 1)[1]
        # if "sliced" in somestring:
        #     string = string.split("minced", 1)[1]
        # if "sliced" in somestring:
        #     string = string.split("crushed", 1)[1]
        # string = string.split("grated", 1)[1]
        if string == "skinless":
            continue
        parsed.append(string)
    parsed = str(parsed).lower().replace("'", "").replace('"','')
    return parsed

recipes['ingredients'] = recipes['ingredients'].apply(lambda x: process_ingredients(x))
recipes.head(30)



,recipe_id,recipe_tag,title,rating,total_reviews,ingredients,calories,carbs,protein,cholesterol,sodium
0,1,italian,Italian Chicken Cacciatore,5.00,3,"[olive oil, onion, all-purpose flour, salt, fr...",587,36.5,38.8,117,1707
1,2,italian,Spaghetti Cacio e Pepe,4.58,73,"[spaghetti, olive oil, garlic, ground black pe...",806,87.8,31.6,54,633
2,3,italian,Easy Caponata,4.67,3,"[stalks stalk celery, olive oil, onion, medium...",289,25.2,4.4,0,424
3,4,italian,Chef John's Lemon Ice,4.64,10,"[cold water, white sugar, lemon zest, freshly ...",94,24.8,0.1,0,4
4,5,italian,Focaccia di Recco,4.60,17,"[all-purpose flour, cold water, cold water, ex...",186,24.1,4.7,7,327
5,6,italian,Frittata di Zucchine e Fiori di Zucca (Italian...,4.50,1,"[extra-virgin olive oil, white onion, bunch pa...",267,10.3,14.5,284,250
6,7,italian,Melanzana alla Parmigiana (Perfect Eggplant Pa...,3.50,1,"[extra-virgin olive oil, eggplants, salt to ta...",506,36.8,28.9,83,453
7,8,italian,Affogato with Cold-Brewed Coffee,4.67,8,"[medium-coarse ground coffee, cold water, vani...",109,15.4,2.1,9,46
8,9,italian,World's Best Lasagna,4.79,11758,"[sweet italian sausage, lean ground beef, oni...",448,36.5,29.7,82,1788
9,10,italian,Chicken Marsala,4.54,3698,"[all-purpose flour for coating, salt, ground b...",448,13.3,28.8,99,543


In [15]:
# write it out to csv
recipes.to_csv("../../data/clean_recipes.csv", index=False)